<a class="anchor" id="0."></a>**Content**

1. [Summary](#1.)
1. [Reading the Data](#2.)
1. [Data Cleaning and Text Preprocessing](#3.)
1. [Creating Features from a Bag of Words](#4.)
1. [Classification using Random Forest](#5.)
1. [Creating a Submission](#6.)

Quick Links: [1](#1.)  |  [2](#2.)  |  [3](#3.)  |  [4](#4.)  |  [5](#5.)  |   [6](#6.)

 [Go to Content Menu](#0.)    |             Quick Links: [1](#1.)  |  [2](#2.)  |  [3](#3.)  |  [4](#4.)  |  [5](#5.)  |   [6](#6.)

<a class="anchor" id="1."></a>
# **1. Summary** 

### Objective
In this notebook we will perform movie review sentiment analysis using Word2Vec.
### About dataset
The labeled data set consists of 50,000 IMDB movie reviews. The sentiment of reviews is binary, meaning:
* the IMDB rating < 5 results in a sentiment score of 0, 
* and rating >=7 have a sentiment score of 1.

 [Go to Content Menu](#0.)    |             Quick Links: [1](#1.)  |  [2](#2.)  |  [3](#3.)  |  [4](#4.)  |  [5](#5.)  |   [6](#6.)

<a class="anchor" id="2."></a>
# **2. Reading the Data** 

Import required libraries

In [ ]:
import pandas as pd 
from bs4 import BeautifulSoup  
import re
from nltk.corpus import stopwords # Import the stop word list

Read data

In [ ]:
train = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0,delimiter="\t", quoting=3)

Review data

In [ ]:
print("Number of rows: ",train.shape[0])
print("Number of columns: ",train.shape[1])
print("List of columns: ",train.columns)


First five rows of the train data

In [ ]:
train.head(5)

Checkout first review in the data

In [ ]:
print(train["review"][0])

 [Go to Content Menu](#0.)    |             Quick Links: [1](#1.)  |  [2](#2.)  |  [3](#3.)  |  [4](#4.)  |  [5](#5.)  |   [6](#6.)

<a class="anchor" id="3."></a>
# **3. Data Cleaning and Text Preprocessing** 

In [ ]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [ ]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["review"].size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
print("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews = []
for i in range( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%10000 == 0 ):
        print("Review %d of %d completed\n" % ( i+1, num_reviews ))                                                                    
    clean_train_reviews.append( review_to_words( train["review"][i] ))
print("Data cleaning completed\n")

 [Go to Content Menu](#0.)    |             Quick Links: [1](#1.)  |  [2](#2.)  |  [3](#3.)  |  [4](#4.)  |  [5](#5.)  |   [6](#6.)

<a class="anchor" id="4."></a>
# **4 Creating Features from a Bag of Words** 

In [ ]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()
print("done\n")

In [ ]:
train_data_features.shape

In [ ]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print(vocab)

In [ ]:
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print(count, tag)

 [Go to Content Menu](#0.)    |             Quick Links: [1](#1.)  |  [2](#2.)  |  [3](#3.)  |  [4](#4.)  |  [5](#5.)  |   [6](#6.)

<a class="anchor" id="5."></a>
# **5. Classification using Random Forest** 

In [ ]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 200) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )
print('done')

 [Go to Content Menu](#0.)    |             Quick Links: [1](#1.)  |  [2](#2.)  |  [3](#3.)  |  [4](#4.)  |  [5](#5.)  |   [6](#6.)

<a class="anchor" id="6."></a>
# **6. Creating a Submission** 

In [ ]:
# Read the test data
test = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print(test.shape)

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 10000 == 0 ):
        print("Review %d of %d completed\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )
print("Output data ready for submission\n")